<a href="https://colab.research.google.com/github/traptisinghh/Projects/blob/main/RAG_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install faiss-cpu

In [6]:
!pip install langchain_community

In [15]:
# Rag with Langchain and FIASS to read external text and PDF

!pip install faiss-cpu
!pip install langchain_community

import os
import faiss
from google.colab import files
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Upload files
uploaded_files = files.upload()

documents = []
for filename in uploaded_files.keys():
    if filename.endswith(".txt"):
        text_loader = TextLoader(filename)
        docs = text_loader.load()
        documents.extend(docs)
    elif filename.endswith(".pdf"):
        pdf_loader = PyPDFLoader(filename)
        docs = pdf_loader.load()
        documents.extend(docs)

# Print original text content
print("\nUploaded Document Contents:\n")
for i, doc in enumerate(documents):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 80)  # Separator for readability


Saving agenticAI_contextEngineering.pdf to agenticAI_contextEngineering (2).pdf

Uploaded Document Contents:

Document 1:
Agentic Context Engineering: Evolving Contexts for Self-Improving
Language Models
Qizheng Zhang 1∗ Changran Hu 2∗ Shubhangi Upasani 2 Boyuan Ma 2 Fenglu Hong 2
Vamsidhar Kamanuru 2 Jay Rainton 2 Chen Wu 2 Mengmeng Ji 2 Hanchen Li 3
Urmish Thakker 2 James Zou 1 Kunle Olukotun 1
1 Stanford University 2 SambaNova Systems, Inc. 3 UC Berkeley ∗ equal contribution
/envel⌢peqizhengz@stanford.edu, changran.hu@sambanovasystems.com
Abstract
Large language model (LLM) applications such as agents and domain-specific reasoning increasingly rely on
context adaptation—modifying inputs with instructions, strategies, or evidence, rather than weight updates.
Prior approaches improve usability but often suffer from brevity bias, which drops domain insights for
concise summaries, and from context collapse, where iterative rewriting erodes details over time. Building on
the adaptive mem

In [9]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 8.3 MB/s eta 0:00:00


In [16]:
# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

# Embed using HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings()
hf_vectors = hf_embeddings.embed_documents([doc.page_content for doc in split_docs])

# Print embeddings
print("\nEmbeddings for Document Chunks:\n")
for i, (doc, emb) in enumerate(zip(split_docs, hf_vectors)):
    print(f"Chunk {i+1}: {doc.page_content}")
    print(f"Embedding {i+1}: {emb[:10]} ... (truncated for readability)")
    print("-" * 80)

print("\nTotal Embeddings Created:", len(hf_vectors))

/tmp/ipython-input-573475107.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hf_embeddings = HuggingFaceEmbeddings()



Embeddings for Document Chunks:

Chunk 1: Agentic Context Engineering: Evolving Contexts for Self-Improving
Language Models
Qizheng Zhang 1∗ Changran Hu 2∗ Shubhangi Upasani 2 Boyuan Ma 2 Fenglu Hong 2
Vamsidhar Kamanuru 2 Jay Rainton 2 Chen Wu 2 Mengmeng Ji 2 Hanchen Li 3
Urmish Thakker 2 James Zou 1 Kunle Olukotun 1
1 Stanford University 2 SambaNova Systems, Inc. 3 UC Berkeley ∗ equal contribution
/envel⌢peqizhengz@stanford.edu, changran.hu@sambanovasystems.com
Abstract
Embedding 1: [0.047672782093286514, -0.011407041922211647, -0.06681617349386215, 0.016286777332425117, -0.027712173759937286, -0.025939559563994408, 0.005531337112188339, -0.016221169382333755, -0.002828251803293824, -0.05339383706450462] ... (truncated for readability)
--------------------------------------------------------------------------------
Chunk 2: Abstract
Large language model (LLM) applications such as agents and domain-specific reasoning increasingly rely on
context adaptation—modifying inputs with instr

In [17]:
import numpy as np
# Create FAISS instance
dimension = len(hf_vectors[0])  # Assuming fixed embedding dimension
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(np.array(hf_vectors, dtype=np.float32))

# Perform similarity search
query = "example search query"
query_embedding = hf_embeddings.embed_query(query)
D, I = faiss_index.search(np.array([query_embedding], dtype=np.float32), k=5)

# Print search results with matching text and embeddings
print("\nTop Matches for Query:\n")
for match_idx in I[0]:
    print(f"Matched Text Chunk: {split_docs[match_idx].page_content}")
    print(f"Matching Embedding: {hf_vectors[match_idx][:10]} ... (truncated)")
    print("-" * 80)


Top Matches for Query:

Matched Text Chunk: }
		
]
}
{
		
"reasoning"
:
	
"The	reflection	identifies	a	pagination	handling	error	where	the	agent	used	an	arbitrary	fixed	range	instead	of	proper	pagination
	
logic.	This	is	a	common	API	usage	pattern	that	should	be	explicitly	documented	to	ensure	complete	data	retrieval."
,
		
"operations"
:
	
[
				
{
						
"type"
:
	
"ADD"
,
						
"section"
:
	
"apis_to_use_for_specific_information"
,
						
"content"
:
	
"About	pagination:	many	APIs	return	items	in	
\"
pages
\"
Matching Embedding: [0.04891050606966019, 0.012242255732417107, -0.010965615510940552, -0.01968436874449253, 0.01509763952344656, 0.011500206775963306, -0.022722920402884483, 0.04312090203166008, 0.005730421748012304, -0.006541923154145479] ... (truncated)
--------------------------------------------------------------------------------
Matched Text Chunk: queries, which surface both effective strategies and recurring pitfalls. The Reflector critiques these traces
to extract l

In [19]:
# Ask a question
query = input("\nEnter your query")
query_embedding = hf_embeddings.embed_query(query)
D, I = faiss_index.search(np.array([query_embedding], dtype=np.float32), k=1)

# Print answer and embedding
best_match_idx = I[0][0]
print("\n Best matching Answer")
print(split_docs[best_match_idx].page_content)
print("\n Corresponding Embedding (Trncated for readability)")
print(hf_vectors[best_match_idx][:10])


Enter your querywhat is this paper about

 Best matching Answer
,
						
"content"
:
	
"[New	calculation	method...]"
				
}
}
		
]
}
Figure 14: ACE Curator prompt on FINER
23

 Corresponding Embedding (Trncated for readability)
[-0.012883717194199562, -0.04099702090024948, -0.021996846422553062, 0.027037998661398888, -0.0502479150891304, 0.0034391358494758606, 0.027777109295129776, 0.026866871863603592, -0.07296739518642426, 0.0576101653277874]
